In [1]:
import pandas as pd
import plotly
import plotly.graph_objects as go

In [92]:
df = pd.read_csv('Backup_List.csv')
tabs = list(df.Tab.unique())

f = open('Backup_Change_Summary.html', "w")
f.write('<link rel="stylesheet" href="style.css">\n')
f.write('<script src="script.js"></script>\n')
f.write('<link rel="stylesheet" href="style.css">\n')
f.write('<!DOCTYPE html>\n')
f.write('<html>\n')
f.write('<head>\n')
f.write('<meta charset="utf-8">\n')
f.write('</head>\n')
f.write('<body>\n\n')

f.write('<div class="tab">\n')
for i, tab in enumerate(tabs):
    if i == 0: #Make tab active
        f.write('  <button class="tablinks active" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
    else:
        f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
f.write('</div>\n')

for i, tab in enumerate(tabs):
    if i == 0: #Make tab active
        f.write('<div id="' + tab + '" class="tabcontent" style="display:block">\n')
    else:
        f.write('<div id="' + tab + '" class="tabcontent">\n')
    df_tab = df[df.Tab==tab]
    

    
    descriptions = df_tab.Description.unique()
    for description in descriptions:
        df_description = df_tab[df_tab.Description==description]
        duplicate_mask = df_description.duplicated(subset=['Backup_Number'], keep='first')
        df_description = df_description[~duplicate_mask]
        df_description = df_description.sort_values(by='Backup_Number')
        df_description['Difference'] = df_description.Value.diff().fillna(method='bfill').astype('int')
        df_description['Update'] = 'No Change'
        df_description.loc[df_description.Difference != 0, 'Update'] = 'Change'
        df_description["Color"] = df_description["Update"].map({"Change": "Crimson","No Change": "Grey"})
        unit = df_description.iloc[0,3] #The same for all rows so just grab the first
        
        fig = go.Figure(data=[go.Bar(
            x=df_description.Backup_Number,
            y=df_description.Value,
            name=description,   
            marker_color=list(df_description.Color) # marker color can be a single color value or an iterable
        )])
        fig.update_layout(title=description)
        

        df_change = df_description[df_description.Update=='Change'].copy()
        for index, row in df_change.iterrows():
            diff_text = str(row['Difference'])
            if row['Difference'] > 0:
                diff_text = '+' + diff_text

            fig.add_annotation(x=row['Backup_Number'], y=row['Value'] + row['Value']/10,
                text=diff_text,
                showarrow=False,
                arrowhead=1)

        
        fig.update_layout(
            autosize=False,
            width = 1700,
            height=250,
            margin=dict(
                l=50,
                r=50,
                b=50,
                t=50,
                pad=4
                ),
            yaxis_title = unit,
            xaxis_title = 'Version Number'
            )


        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
            
    f.write('</div>\n')
        
f.write('</body>\n')
f.write('</html>\n')
f.close()
        